In [1]:
import numpy as np
import pandas as pd
import time
import os

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'dask-worker-space',
 'MergedWithLabels.csv',
 'modeling_anom.ipynb',
 'modeling_anom_concise.ipynb',
 'pd_anom_part_a_1',
 'prvdr_type.csv',
 'Rndrng_Prvdr_Type_2modified.csv',
 'Rndrng_Prvdr_Type_modified.csv',
 'test2_anom.ipynb',
 'text_anom.py',
 'y_train_abod.txt']

## Exploring dataset.  


## Read in Dataset

In [4]:
filename=os.path.join(os.getcwd(), 'MergedWithLabels.csv')
filename

'/mnt/c/Users/John Olanipekun/Documents/Machine Learning/SMU_Machine_Learning/capstone2/MergedWithLabels.csv'

### Trying PySpark

In [50]:
import pyspark
sc = pyspark.sql.SparkSession.builder.appName("MyApp") \
            .master("local")\
            .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.9.5") \
            .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven") \
            .getOrCreate()


In [49]:
#sc.stop()

In [5]:
from pyspark.sql import SparkSession
sc = SparkSession.builder.master("local").appName("Test").getOrCreate()

22/02/25 10:21:05 WARN Utils: Your hostname, eagle-john resolves to a loopback address: 127.0.1.1; using 172.26.244.221 instead (on interface eth0)
22/02/25 10:21:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/25 10:21:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
raw_data = sc.read.options(delimiter=",",header=True).csv(filename)
#raw_data=sc.read.csv(filename,header=True, inferSchema=True, sep='\t')


In [7]:
type(raw_data)

pyspark.sql.dataframe.DataFrame

In [8]:
raw_data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Unnamed: 0: string (nullable = true)
 |-- Rndrng_NPI: string (nullable = true)
 |-- Rndrng_Prvdr_Type: string (nullable = true)
 |-- Rndrng_Prvdr_Gndr: string (nullable = true)
 |-- Year_x: string (nullable = true)
 |-- HCPCS_Cd_count_x: string (nullable = true)
 |-- HCPCS_Cd_nunique_x: string (nullable = true)
 |-- Place_Of_Srvc_count: string (nullable = true)
 |-- Place_Of_Srvc_nunique: string (nullable = true)
 |-- Tot_Benes_mean: string (nullable = true)
 |-- Tot_Benes_min: string (nullable = true)
 |-- Tot_Benes_max: string (nullable = true)
 |-- Tot_Benes_median: string (nullable = true)
 |-- Tot_Benes_std: string (nullable = true)
 |-- Tot_Srvcs_mean: string (nullable = true)
 |-- Tot_Srvcs_min: string (nullable = true)
 |-- Tot_Srvcs_max: string (nullable = true)
 |-- Tot_Srvcs_median: string (nullable = true)
 |-- Tot_Srvcs_std: string (nullable = true)
 |-- Tot_Bene_Day_Srvcs_mean: string (nullable = true)
 |-- Tot_Bene_Day_Srvcs_m

In [9]:
raw_data.show(n=10)

22/02/25 10:21:20 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/02/25 10:21:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, Rndrng_NPI, Rndrng_Prvdr_Type, Rndrng_Prvdr_Gndr, Year_x, HCPCS_Cd_count_x, HCPCS_Cd_nunique_x, Place_Of_Srvc_count, Place_Of_Srvc_nunique, Tot_Benes_mean, Tot_Benes_min, Tot_Benes_max, Tot_Benes_median, Tot_Benes_std, Tot_Srvcs_mean, Tot_Srvcs_min, Tot_Srvcs_max, Tot_Srvcs_median, Tot_Srvcs_std, Tot_Bene_Day_Srvcs_mean, Tot_Bene_Day_Srvcs_min, Tot_Bene_Day_Srvcs_max, Tot_Bene_Day_Srvcs_median, Tot_Bene_Day_Srvcs_std, Avg_Sbmtd_Chrg_mean, Avg_Sbmtd_Chrg_min, Avg_Sbmtd_Chrg_max, Avg_Sbmtd_Chrg_median, Avg_Sbmtd_Chrg_std, Avg_Mdcr_Pymt_Amt_mean, Avg_Mdcr_Pymt_Amt_min, Avg_Mdcr_Pymt_Amt_max, Avg_Mdcr_Pymt_Amt_median, Avg_Mdcr_Pymt_Amt_std, key, Brnd_Name_count, Brnd_Name_nunique, Tot_Clms_min, Tot_Clms_max,

+---+----------+----------+--------------------+-----------------+------+----------------+------------------+-------------------+---------------------+------------------+-------------+-------------+----------------+------------------+------------------+-------------+-------------+----------------+------------------+-----------------------+----------------------+----------------------+-------------------------+----------------------+-------------------+------------------+------------------+---------------------+------------------+----------------------+---------------------+---------------------+------------------------+---------------------+--------------------+---------------+-----------------+------------+------------+------------------+---------------+------------------+-------------------+-------------------+--------------------+----------------------+-------------------+-----------------+-----------------+------------------+--------------------+------------------+----------------+

In [10]:
raw_data.columns

['_c0',
 'Unnamed: 0',
 'Rndrng_NPI',
 'Rndrng_Prvdr_Type',
 'Rndrng_Prvdr_Gndr',
 'Year_x',
 'HCPCS_Cd_count_x',
 'HCPCS_Cd_nunique_x',
 'Place_Of_Srvc_count',
 'Place_Of_Srvc_nunique',
 'Tot_Benes_mean',
 'Tot_Benes_min',
 'Tot_Benes_max',
 'Tot_Benes_median',
 'Tot_Benes_std',
 'Tot_Srvcs_mean',
 'Tot_Srvcs_min',
 'Tot_Srvcs_max',
 'Tot_Srvcs_median',
 'Tot_Srvcs_std',
 'Tot_Bene_Day_Srvcs_mean',
 'Tot_Bene_Day_Srvcs_min',
 'Tot_Bene_Day_Srvcs_max',
 'Tot_Bene_Day_Srvcs_median',
 'Tot_Bene_Day_Srvcs_std',
 'Avg_Sbmtd_Chrg_mean',
 'Avg_Sbmtd_Chrg_min',
 'Avg_Sbmtd_Chrg_max',
 'Avg_Sbmtd_Chrg_median',
 'Avg_Sbmtd_Chrg_std',
 'Avg_Mdcr_Pymt_Amt_mean',
 'Avg_Mdcr_Pymt_Amt_min',
 'Avg_Mdcr_Pymt_Amt_max',
 'Avg_Mdcr_Pymt_Amt_median',
 'Avg_Mdcr_Pymt_Amt_std',
 'key',
 'Brnd_Name_count',
 'Brnd_Name_nunique',
 'Tot_Clms_min',
 'Tot_Clms_max',
 'Tot_Clms_mean',
 'Tot_Clms_median',
 'Tot_Clms_std',
 'Tot_30day_Fills_min',
 'Tot_30day_Fills_max',
 'Tot_30day_Fills_mean',
 'Tot_30day_Fills_med

In [11]:
from pyspark.sql.functions import col


## Explore the dataset

## Break the dataset in to three parts.


## Merge provider types with similar names  


In [12]:
raw_data=raw_data.withColumn('Rndrng_Prvdr_Type_2', raw_data['Rndrng_Prvdr_Type'])

In [13]:
raw_data.columns

['_c0',
 'Unnamed: 0',
 'Rndrng_NPI',
 'Rndrng_Prvdr_Type',
 'Rndrng_Prvdr_Gndr',
 'Year_x',
 'HCPCS_Cd_count_x',
 'HCPCS_Cd_nunique_x',
 'Place_Of_Srvc_count',
 'Place_Of_Srvc_nunique',
 'Tot_Benes_mean',
 'Tot_Benes_min',
 'Tot_Benes_max',
 'Tot_Benes_median',
 'Tot_Benes_std',
 'Tot_Srvcs_mean',
 'Tot_Srvcs_min',
 'Tot_Srvcs_max',
 'Tot_Srvcs_median',
 'Tot_Srvcs_std',
 'Tot_Bene_Day_Srvcs_mean',
 'Tot_Bene_Day_Srvcs_min',
 'Tot_Bene_Day_Srvcs_max',
 'Tot_Bene_Day_Srvcs_median',
 'Tot_Bene_Day_Srvcs_std',
 'Avg_Sbmtd_Chrg_mean',
 'Avg_Sbmtd_Chrg_min',
 'Avg_Sbmtd_Chrg_max',
 'Avg_Sbmtd_Chrg_median',
 'Avg_Sbmtd_Chrg_std',
 'Avg_Mdcr_Pymt_Amt_mean',
 'Avg_Mdcr_Pymt_Amt_min',
 'Avg_Mdcr_Pymt_Amt_max',
 'Avg_Mdcr_Pymt_Amt_median',
 'Avg_Mdcr_Pymt_Amt_std',
 'key',
 'Brnd_Name_count',
 'Brnd_Name_nunique',
 'Tot_Clms_min',
 'Tot_Clms_max',
 'Tot_Clms_mean',
 'Tot_Clms_median',
 'Tot_Clms_std',
 'Tot_30day_Fills_min',
 'Tot_30day_Fills_max',
 'Tot_30day_Fills_mean',
 'Tot_30day_Fills_med

In [14]:
from pyspark.sql.functions import *

In [15]:
newDf = raw_data.withColumn(
    "Rndrng_Prvdr_Type_2",
    when(
        col("Rndrng_Prvdr_Type_2").like("%CRNA%"),
        lit("Certified Registered Nurse Anesthetist")
    ).otherwise(col("Rndrng_Prvdr_Type_2"))    
    
)

In [16]:

newDf = newDf.withColumn(
    "Rndrng_Prvdr_Type_2",
    when(
        col("Rndrng_Prvdr_Type_2").like("%Hematology%"),
        lit("Hematology")
    ).otherwise(col("Rndrng_Prvdr_Type_2"))    
    
)

In [17]:
newDf = newDf.withColumn(
    "Rndrng_Prvdr_Type_2",
    when(
        col("Rndrng_Prvdr_Type_2").like("%Audiologist%"),
        lit("Audiologist")
    ).otherwise(col("Rndrng_Prvdr_Type_2"))    
    
)

In [18]:
newDf = newDf.withColumn(
    "Rndrng_Prvdr_Type_2",
    when(
        col("Rndrng_Prvdr_Type_2").like("%Pain Management%"),
        lit("Pain Management")
    ).otherwise(col("Rndrng_Prvdr_Type_2"))    
    
)

In [19]:
newDf = newDf.withColumn(
    "Rndrng_Prvdr_Type_2",
    when(
        col("Rndrng_Prvdr_Type_2").like("%Cardiology%"),
        lit("Cardiology")
    ).otherwise(col("Rndrng_Prvdr_Type_2"))    
    
)

In [20]:
newDf = newDf.withColumn(
    "Rndrng_Prvdr_Type_2",
    when(
        col("Rndrng_Prvdr_Type_2").like("%Colorectal%"),
        lit("Colorectal Surgery")
    ).otherwise(col("Rndrng_Prvdr_Type_2"))    
    
)

In [21]:
newDf = newDf.withColumn(
    "Rndrng_Prvdr_Type_2",
    when(
        col("Rndrng_Prvdr_Type_2").like("%Oral%"),
        lit("Oral Surgery")
    ).otherwise(col("Rndrng_Prvdr_Type_2"))    
    
)

In [22]:
newDf = newDf.withColumn("Rndrng_Prvdr_Type_2", regexp_replace("Rndrng_Prvdr_Type_2", "Obstetrics & Gynecology", "Obstetrics/Gynecology"))
newDf = newDf.withColumn("Rndrng_Prvdr_Type_2", regexp_replace("Rndrng_Prvdr_Type_2","Psychologist, Clinical", "Clinical Psychologist"))
newDf = newDf.withColumn("Rndrng_Prvdr_Type_2", regexp_replace("Rndrng_Prvdr_Type_2","Gynecological Oncology", "Gynecological/Oncology"))
newDf = newDf.withColumn("Rndrng_Prvdr_Type_2", regexp_replace("Rndrng_Prvdr_Type_2","Occupational Therapist in Private Practice", "Occupational therapist"))
newDf = newDf.withColumn("Rndrng_Prvdr_Type_2", regexp_replace("Rndrng_Prvdr_Type_2","Allergy/ Immunology", "Allergy/Immunology"))
newDf = newDf.withColumn("Rndrng_Prvdr_Type_2", regexp_replace("Rndrng_Prvdr_Type_2","Occupational Therapist in Private Practice", "Occupational therapist"))



In [23]:
#review the result in excel

newDf.select("Rndrng_Prvdr_Type").distinct().toPandas().to_csv('Rndrng_Prvdr_Type_modified.csv')
newDf.select("Rndrng_Prvdr_Type_2").distinct().toPandas().to_csv('Rndrng_Prvdr_Type_2modified.csv')


In [24]:
newDf2 = newDf.alias('newDf2')

In [25]:
newDf2.columns

['_c0',
 'Unnamed: 0',
 'Rndrng_NPI',
 'Rndrng_Prvdr_Type',
 'Rndrng_Prvdr_Gndr',
 'Year_x',
 'HCPCS_Cd_count_x',
 'HCPCS_Cd_nunique_x',
 'Place_Of_Srvc_count',
 'Place_Of_Srvc_nunique',
 'Tot_Benes_mean',
 'Tot_Benes_min',
 'Tot_Benes_max',
 'Tot_Benes_median',
 'Tot_Benes_std',
 'Tot_Srvcs_mean',
 'Tot_Srvcs_min',
 'Tot_Srvcs_max',
 'Tot_Srvcs_median',
 'Tot_Srvcs_std',
 'Tot_Bene_Day_Srvcs_mean',
 'Tot_Bene_Day_Srvcs_min',
 'Tot_Bene_Day_Srvcs_max',
 'Tot_Bene_Day_Srvcs_median',
 'Tot_Bene_Day_Srvcs_std',
 'Avg_Sbmtd_Chrg_mean',
 'Avg_Sbmtd_Chrg_min',
 'Avg_Sbmtd_Chrg_max',
 'Avg_Sbmtd_Chrg_median',
 'Avg_Sbmtd_Chrg_std',
 'Avg_Mdcr_Pymt_Amt_mean',
 'Avg_Mdcr_Pymt_Amt_min',
 'Avg_Mdcr_Pymt_Amt_max',
 'Avg_Mdcr_Pymt_Amt_median',
 'Avg_Mdcr_Pymt_Amt_std',
 'key',
 'Brnd_Name_count',
 'Brnd_Name_nunique',
 'Tot_Clms_min',
 'Tot_Clms_max',
 'Tot_Clms_mean',
 'Tot_Clms_median',
 'Tot_Clms_std',
 'Tot_30day_Fills_min',
 'Tot_30day_Fills_max',
 'Tot_30day_Fills_mean',
 'Tot_30day_Fills_med

In [26]:
drop_cols = ('_c0', 'Unnamed: 0', 'EXCL Year', 'EXCLDATE', 'EXCLTYPE', 'NPI')

In [27]:
newDf2=newDf2.drop(*drop_cols)

In [28]:
newDf2=newDf2.drop('Rndrng_Prvdr_Type')

In [29]:
newDf2.printSchema()

root
 |-- Rndrng_NPI: string (nullable = true)
 |-- Rndrng_Prvdr_Gndr: string (nullable = true)
 |-- Year_x: string (nullable = true)
 |-- HCPCS_Cd_count_x: string (nullable = true)
 |-- HCPCS_Cd_nunique_x: string (nullable = true)
 |-- Place_Of_Srvc_count: string (nullable = true)
 |-- Place_Of_Srvc_nunique: string (nullable = true)
 |-- Tot_Benes_mean: string (nullable = true)
 |-- Tot_Benes_min: string (nullable = true)
 |-- Tot_Benes_max: string (nullable = true)
 |-- Tot_Benes_median: string (nullable = true)
 |-- Tot_Benes_std: string (nullable = true)
 |-- Tot_Srvcs_mean: string (nullable = true)
 |-- Tot_Srvcs_min: string (nullable = true)
 |-- Tot_Srvcs_max: string (nullable = true)
 |-- Tot_Srvcs_median: string (nullable = true)
 |-- Tot_Srvcs_std: string (nullable = true)
 |-- Tot_Bene_Day_Srvcs_mean: string (nullable = true)
 |-- Tot_Bene_Day_Srvcs_min: string (nullable = true)
 |-- Tot_Bene_Day_Srvcs_max: string (nullable = true)
 |-- Tot_Bene_Day_Srvcs_median: string (nul

In [30]:
newDf2.limit(5).toPandas()

,Rndrng_NPI,Rndrng_Prvdr_Gndr,Year_x,HCPCS_Cd_count_x,HCPCS_Cd_nunique_x,Place_Of_Srvc_count,Place_Of_Srvc_nunique,Tot_Benes_mean,Tot_Benes_min,Tot_Benes_max,...,Avg_Suplr_Sbmtd_Chrg_std,Avg_Suplr_Mdcr_Pymt_Amt_min,Avg_Suplr_Mdcr_Pymt_Amt_max,Avg_Suplr_Mdcr_Pymt_Amt_mean,Avg_Suplr_Mdcr_Pymt_Amt_median,Avg_Suplr_Mdcr_Pymt_Amt_std,HCPCS_CD_count,HCPCS_CD_nunique,key2,Rndrng_Prvdr_Type_2
0,1003000126,M,2013,7,7,7,1,158.85714285714286,47,381,...,206.10226545550853,23.320416666999996,139.35321429,81.3368154785,81.3368154785,82.04757803926962,0.0,0.0,1003000126_2013,Internal Medicine
1,1003000134,M,2013,7,7,7,1,691.5714285714287,46,3770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1003000134_2013,Pathology
2,1003000142,M,2013,8,8,8,2,31.0,12,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1003000142_2013,Anesthesiology
3,1003000407,M,2013,21,21,21,1,49.28571428571429,11,181,...,28.433099377290677,0.033148633500000003,36.895454545,15.9824329435,11.018695652,18.92581346210838,0.0,0.0,1003000407_2013,Family Practice
4,1003000423,F,2013,4,4,4,1,22.5,16,29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1003000423_2013,Obstetrics/Gynecology


## OnehotEncoding
https://medium.com/@nutanbhogendrasharma/role-of-onehotencoder-and-pipelines-in-pyspark-ml-feature-part-2-3275767e74f0

In [31]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline

In [32]:
newDf2.columns[80:]

['Avg_Suplr_Sbmtd_Chrg_min',
 'Avg_Suplr_Sbmtd_Chrg_max',
 'Avg_Suplr_Sbmtd_Chrg_mean',
 'Avg_Suplr_Sbmtd_Chrg_median',
 'Avg_Suplr_Sbmtd_Chrg_std',
 'Avg_Suplr_Mdcr_Pymt_Amt_min',
 'Avg_Suplr_Mdcr_Pymt_Amt_max',
 'Avg_Suplr_Mdcr_Pymt_Amt_mean',
 'Avg_Suplr_Mdcr_Pymt_Amt_median',
 'Avg_Suplr_Mdcr_Pymt_Amt_std',
 'HCPCS_CD_count',
 'HCPCS_CD_nunique',
 'key2',
 'Rndrng_Prvdr_Type_2']

### one hot encoding in pyspark
https://medium.com/@nutanbhogendrasharma/role-of-onehotencoder-and-pipelines-in-pyspark-ml-feature-part-2-3275767e74f0

https://stackoverflow.com/questions/35804755/apply-onehotencoder-for-several-categorical-columns-in-sparkmlib


In [33]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderModel

In [34]:
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderModel
# The index of string vlaues multiple columns
categorical_columns= ['Rndrng_Prvdr_Gndr','Rndrng_Prvdr_Type_2','Rfrg_Prvdr_Gndr']

indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c))
    for c in categorical_columns
]


encoder = OneHotEncoder(
    inputCols=[indexer.getOutputCol() for indexer in indexers],
    outputCols=[
        "{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers]
)

### combines features into a single column. So I don't need this.
assembler = VectorAssembler(
    inputCols=encoder.getOutputCols(),
    outputCol="features"
)

pipeline = Pipeline(stages=indexers + [encoder, assembler])
transformed=pipeline.fit(newDf2).transform(newDf2)
transformed.columns[80:]


['Avg_Suplr_Sbmtd_Chrg_min',
 'Avg_Suplr_Sbmtd_Chrg_max',
 'Avg_Suplr_Sbmtd_Chrg_mean',
 'Avg_Suplr_Sbmtd_Chrg_median',
 'Avg_Suplr_Sbmtd_Chrg_std',
 'Avg_Suplr_Mdcr_Pymt_Amt_min',
 'Avg_Suplr_Mdcr_Pymt_Amt_max',
 'Avg_Suplr_Mdcr_Pymt_Amt_mean',
 'Avg_Suplr_Mdcr_Pymt_Amt_median',
 'Avg_Suplr_Mdcr_Pymt_Amt_std',
 'HCPCS_CD_count',
 'HCPCS_CD_nunique',
 'key2',
 'Rndrng_Prvdr_Type_2',
 'Rndrng_Prvdr_Gndr_indexed',
 'Rndrng_Prvdr_Type_2_indexed',
 'Rfrg_Prvdr_Gndr_indexed',
 'Rndrng_Prvdr_Gndr_indexed_encoded',
 'Rndrng_Prvdr_Type_2_indexed_encoded',
 'Rfrg_Prvdr_Gndr_indexed_encoded',
 'features']

In [35]:
transformed.columns[80:]

['Avg_Suplr_Sbmtd_Chrg_min',
 'Avg_Suplr_Sbmtd_Chrg_max',
 'Avg_Suplr_Sbmtd_Chrg_mean',
 'Avg_Suplr_Sbmtd_Chrg_median',
 'Avg_Suplr_Sbmtd_Chrg_std',
 'Avg_Suplr_Mdcr_Pymt_Amt_min',
 'Avg_Suplr_Mdcr_Pymt_Amt_max',
 'Avg_Suplr_Mdcr_Pymt_Amt_mean',
 'Avg_Suplr_Mdcr_Pymt_Amt_median',
 'Avg_Suplr_Mdcr_Pymt_Amt_std',
 'HCPCS_CD_count',
 'HCPCS_CD_nunique',
 'key2',
 'Rndrng_Prvdr_Type_2',
 'Rndrng_Prvdr_Gndr_indexed',
 'Rndrng_Prvdr_Type_2_indexed',
 'Rfrg_Prvdr_Gndr_indexed',
 'Rndrng_Prvdr_Gndr_indexed_encoded',
 'Rndrng_Prvdr_Type_2_indexed_encoded',
 'Rfrg_Prvdr_Gndr_indexed_encoded',
 'features']

In [36]:
#checkpoint here
newDf3 = transformed.alias('newDf3')

## Drop parent and other intermediate columns.

In [37]:
drop_cols2 = ['Rndrng_Prvdr_Gndr','Rfrg_Prvdr_Gndr','Rndrng_Prvdr_Type_2','Rndrng_Prvdr_Gndr_indexed','Rndrng_Prvdr_Type_2_indexed','Rfrg_Prvdr_Gndr_indexed','features']

In [38]:
newDf3=newDf3.drop(*drop_cols2)

In [39]:
### NPI looks like a unique ID that needs to be dropped
newDf3=newDf3.drop('Rndrng_NPI')

In [40]:
newDf3.columns

['Year_x',
 'HCPCS_Cd_count_x',
 'HCPCS_Cd_nunique_x',
 'Place_Of_Srvc_count',
 'Place_Of_Srvc_nunique',
 'Tot_Benes_mean',
 'Tot_Benes_min',
 'Tot_Benes_max',
 'Tot_Benes_median',
 'Tot_Benes_std',
 'Tot_Srvcs_mean',
 'Tot_Srvcs_min',
 'Tot_Srvcs_max',
 'Tot_Srvcs_median',
 'Tot_Srvcs_std',
 'Tot_Bene_Day_Srvcs_mean',
 'Tot_Bene_Day_Srvcs_min',
 'Tot_Bene_Day_Srvcs_max',
 'Tot_Bene_Day_Srvcs_median',
 'Tot_Bene_Day_Srvcs_std',
 'Avg_Sbmtd_Chrg_mean',
 'Avg_Sbmtd_Chrg_min',
 'Avg_Sbmtd_Chrg_max',
 'Avg_Sbmtd_Chrg_median',
 'Avg_Sbmtd_Chrg_std',
 'Avg_Mdcr_Pymt_Amt_mean',
 'Avg_Mdcr_Pymt_Amt_min',
 'Avg_Mdcr_Pymt_Amt_max',
 'Avg_Mdcr_Pymt_Amt_median',
 'Avg_Mdcr_Pymt_Amt_std',
 'key',
 'Brnd_Name_count',
 'Brnd_Name_nunique',
 'Tot_Clms_min',
 'Tot_Clms_max',
 'Tot_Clms_mean',
 'Tot_Clms_median',
 'Tot_Clms_std',
 'Tot_30day_Fills_min',
 'Tot_30day_Fills_max',
 'Tot_30day_Fills_mean',
 'Tot_30day_Fills_median',
 'Tot_30day_Fills_std',
 'Tot_Day_Suply_min',
 'Tot_Day_Suply_max',
 'Tot_Da

In [ ]:
# !spark-shell --packages com.microsoft.azure:synapseml_2.12:0.9.5
# !pyspark --packages com.microsoft.azure:synapseml_2.12:0.9.5
# !spark-submit --packages com.microsoft.azure:synapseml_2.12:0.9.5 MyApp.jar

## Modeling

In [41]:
import pyspark
sc = pyspark.sql.SparkSession.builder.appName("MyApp") \
            .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.9.5") \
            .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven") \
            .getOrCreate()


In [42]:
import synapse.ml

In [46]:
from synapse.ml.isolationforest import *

isolationForest = (IsolationForest()
      .setNumEstimators(100)
      .setBootstrap(False)
      .setMaxSamples(256)
      .setMaxFeatures(1.0)
      .setFeaturesCol("features")
      .setPredictionCol("predictedLabel")
      .setScoreCol("outlierScore")
      .setContamination(0.02)
      .setContaminationError(0.02 * 0.01)
      .setRandomSeed(1))

TypeError: 'JavaPackage' object is not callable